In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
# (6) run `make pipeline` to clean DB
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_33865/564014651.py:12: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104863,167997,https://gelliottmorris.substack.com/p/how-to-r...,How to read outlier polls,\N,\N,2023-09-29 10:44:38.637712+00,2023-09-29 10:44:38.637712+00,NaN,\N,GEMorris,51.0,0.0,\N
104864,167998,https://www.jefftk.com/p/weighing-animal-worth,Weighing Animal Worth,\N,\N,2023-09-29 10:44:38.639582+00,2023-09-29 10:44:38.639582+00,NaN,\N,JeffKauffman,2.0,0.0,\N
104865,167999,https://blog.aiimpacts.org/p/my-current-though...,My Current Thoughts on the AI Strategic Landscape,\N,\N,2023-09-29 10:44:38.641431+00,2023-09-29 10:44:38.641431+00,NaN,\N,AI Impacts,74.0,0.0,\N
104866,168000,https://www.askamanager.org/2023/09/how-do-you...,how do you find a lawyer for workplace issues?,\N,\N,2023-09-29 10:44:38.643238+00,2023-09-29 10:44:38.643238+00,NaN,\N,AskManager,32.0,0.0,\N


In [2]:
links['added'].max()

'2023-09-29 10:44:38.645023+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
104846    167920
104847    167921
104848    167922
104849    167923
104850    167924
Name: id, Length: 13415, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
104862,167996,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-09-29 10:44:38.635854+00,2023-09-29 10:44:38.635854+00,NaN,\N,CEA,13.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

104862    167996
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/22
...11/22
...21/22
...22/22


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104885,168019,https://docs.google.com/document/d/1mCitTPwVhU...,Rethink Priorities’ 2023 Impact 2024 Strategy ...,\N,\N,2023-09-29 19:51:05.862301,2023-09-29 19:51:05.862306,\N,\N,Custom,14.0,0.0,\N
104886,168020,https://docs.google.com/document/d/1sKWqs0zVuH...,CPR – research and founder search plan - Googl...,\N,\N,2023-09-29 19:51:05.868949,2023-09-29 19:51:05.868955,\N,\N,Custom,40.0,0.0,\N
104887,168021,https://www.wired.com/story/fast-forward-elon-...,Six Months Ago Elon Musk Called for a Pause on...,\N,\N,2023-09-29 19:51:05.875868,2023-09-29 19:51:05.875876,\N,\N,Custom,18.0,0.0,\N
104888,168022,https://www.technologyreview.com/2023/09/26/10...,Six months on from the “pause” letter,\N,\N,2023-09-29 19:51:05.882080,2023-09-29 19:51:05.882085,\N,\N,Custom,61.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104885,168019,https://docs.google.com/document/d/1mCitTPwVhU...,Rethink Priorities’ 2023 Impact 2024 Strategy ...,\N,\N,2023-09-29 19:51:05.862301,2023-09-29 19:51:05.862306,\N,\N,Custom,14.0,0.0,\N
104886,168020,https://docs.google.com/document/d/1sKWqs0zVuH...,CPR – research and founder search plan - Googl...,\N,\N,2023-09-29 19:51:05.868949,2023-09-29 19:51:05.868955,\N,\N,Custom,40.0,0.0,\N
104887,168021,https://www.wired.com/story/fast-forward-elon-...,Six Months Ago Elon Musk Called for a Pause on...,\N,\N,2023-09-29 19:51:05.875868,2023-09-29 19:51:05.875876,\N,\N,Custom,18.0,0.0,\N
104888,168022,https://www.technologyreview.com/2023/09/26/10...,Six months on from the “pause” letter,\N,\N,2023-09-29 19:51:05.882080,2023-09-29 19:51:05.882085,\N,\N,Custom,61.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104885,168019,https://docs.google.com/document/d/1mCitTPwVhU...,Rethink Priorities’ 2023 Impact 2024 Strategy ...,\N,\N,2023-09-29 19:51:05.862301,2023-09-29 19:51:05.862306,\N,\N,Custom,14,0,\N
104886,168020,https://docs.google.com/document/d/1sKWqs0zVuH...,CPR – research and founder search plan - Googl...,\N,\N,2023-09-29 19:51:05.868949,2023-09-29 19:51:05.868955,\N,\N,Custom,40,0,\N
104887,168021,https://www.wired.com/story/fast-forward-elon-...,Six Months Ago Elon Musk Called for a Pause on...,\N,\N,2023-09-29 19:51:05.875868,2023-09-29 19:51:05.875876,\N,\N,Custom,18,0,\N
104888,168022,https://www.technologyreview.com/2023/09/26/10...,Six months on from the “pause” letter,\N,\N,2023-09-29 19:51:05.882080,2023-09-29 19:51:05.882085,\N,\N,Custom,61,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)